In [ ]:
%load_ext autoreload
%autoreload 2
%cd /mnt/c/Users/resha/Documents/Github/balancing_framework/

import os
import pickle
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd

import time
import optuna
import argparse


from gluonts.evaluation import make_evaluation_predictions
from gluonts.mx.trainer import Trainer
from gluonts.mx.trainer.callback import TrainingHistory
from gluonts.mx.distribution import StudentTOutput, MultivariateGaussianOutput
from sklearn.metrics import mean_absolute_error
from gluonts.dataset.multivariate_grouper import MultivariateGrouper

from gluonts.mx.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.mx.model.transformer import TransformerEstimator
from gluonts.mx.model.deepar import DeepAREstimator 
from gluonts.mx.model.wavenet import WaveNetEstimator
from gluonts.mx.model.seq2seq import MQCNNEstimator

import matplotlib.pyplot as plt
import json
from pathlib import Path
from gluonts.dataset.split import split
from gluonts.dataset.common import ListDataset
import copy

from gluonts.evaluation import Evaluator
from gluonts.evaluation import make_evaluation_predictions

from gluonts_utils import series_to_gluonts_dataset, load_params

In [ ]:

class Objective:
    def __init__( self, model, dataset_name, X):
        self.model = model
        self.dataset_name = dataset_name
        self.data_params = load_params('gluonts_params.txt', dataset_name)

        predlen = self.data_params['prediction_length']
        X_train_val, X_test = X[:-predlen], X[-predlen:]
        X_train, X_val = X_train_val[:-predlen], X_train_val[-predlen:]
        
        self.tuning_dataset = series_to_gluonts_dataset(X_train, X_val,  self.data_params)
        self.eval_dataset =  series_to_gluonts_dataset(X_train_val, X_test,  self.data_params)
        

    def get_params(self, trial) -> dict:
        if self.model == 'feedforward':
            return {
              "num_hidden_dimensions": [trial.suggest_int("hidden_dim_{}".format(i), 10, 100) for i in range(trial.suggest_int("num_layers", 1, 5))],
              "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
              "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        elif self.model == 'wavenet':
            return {
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        elif self.model == 'mqcnn':
            return {
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        elif self.model == 'deepar':
            return {
                "num_cells": trial.suggest_int("num_cells", 10, 100),
                "num_layers": trial.suggest_int("num_layers", 1, 10),
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100)
            }
        elif self.model == 'transformer':
            # num_heads must divide model_dim
            valid_pairs = [ (i,d) for i in range(10,101) for d in range(1,11) if i%d == 0  ]
            model_dim_num_heads_pair = trial.suggest_categorical("model_dim_num_heads_pair", valid_pairs)

            return {
                "inner_ff_dim_scale": trial.suggest_int("inner_ff_dim_scale", 1, 5),
                "model_dim": model_dim_num_heads_pair[0],
                "embedding_dimension": trial.suggest_int("embedding_dimension", 1, 10),
                "num_heads": model_dim_num_heads_pair[1],
                "dropout_rate": trial.suggest_uniform("dropout_rate", 0.0, 0.5),
                "trainer:learning_rate": trial.suggest_loguniform("trainer:learning_rate", 1e-6, 1e-3),
                "trainer:epochs": trial.suggest_int("trainer:epochs", 10, 100),
            }
        
    def load_model(self, params):
        history = TrainingHistory()
        if self.model == 'feedforward':
            estimator = SimpleFeedForwardEstimator(
                num_hidden_dimensions= params['num_hidden_dimensions'], #num_hidden_dimensions,
                prediction_length=self.prediction_length,
                context_length=self.context_length,
                batch_normalization=False,
                mean_scaling=False,
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history]),
            )
        elif self.model == 'wavenet':
            estimator = WaveNetEstimator(
                freq=self.freq,
                prediction_length=self.prediction_length,
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                    num_batches_per_epoch=100, callbacks=[history], add_default_callbacks=False),
            )
        elif self.model == 'mqcnn':
            estimator = MQCNNEstimator(
                freq=self.freq,
                prediction_length=self.prediction_length,
                context_length=self.context_length,
                distr_output=StudentTOutput(),
                quantiles=None,
                scaling=False, 
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history], hybridize=False),
            )
        elif self.model == 'deepar':
            estimator = DeepAREstimator(
                freq=self.freq,
                context_length=self.context_length,
                distr_output=StudentTOutput(),
                prediction_length=self.prediction_length,
                # num_cells= params['num_cells'],
                # num_layers= params['num_layers'],
                scaling=False, # True by default
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history]),
            )
        elif self.model == 'transformer':
            estimator = TransformerEstimator(
                freq=self.freq,
                context_length=self.context_length,
                prediction_length=self.prediction_length,
                distr_output=StudentTOutput(),
                inner_ff_dim_scale= params['inner_ff_dim_scale'],
                model_dim= params['model_dim'],
                embedding_dimension= params['embedding_dimension'],
                num_heads= params['num_heads'],
                dropout_rate= params['dropout_rate'],
                # scaling=False, # True by default False
                trainer=Trainer(ctx=self.ctx,epochs=params['trainer:epochs'], learning_rate=params['trainer:learning_rate'],
                                num_batches_per_epoch=100, callbacks=[history]),
            )

        return estimator, history

    def train_test(self, params, tuning=True):
        model, history = self.load_model(params)

        if tuning:
            predictor = model.train(self.tuning_dataset.train, self.tuning_dataset.test)
            forecast_it, ts_it = make_evaluation_predictions(
                dataset=self.tuning_dataset.test,
                predictor=predictor,
            )
        else:
            predictor = model.train(self.eval_dataset.train)
            forecast_it, ts_it = make_evaluation_predictions(
                dataset=self.eval_dataset.test,
                predictor=predictor,
            )

        forecasts = list(forecast_it)
        tss = list(ts_it)
        evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
        agg_metrics, item_metrics = evaluator(tss, forecasts)
        
        print(f'#####__tuning mase = {agg_metrics["MASE"]}__###### ')
        return agg_metrics, predictor, history
        

    def __call__(self, trial):

        params = self.get_params(trial)

        agg_metrics, _, _ = self.train_test(params, tuning=True)

        return agg_metrics['MASE']

In [ ]:
X = pd.read_csv('m4_1165.csv')
X

In [ ]:
dataset_name = 'm4_daily_dataset'
model = 'transformer'
save_label = 'original'
n_trials = 1 #10
n_repeats = 1 #5
X = pd.read_csv('m4_1165.csv')


start_time = time.perf_counter()

# run tuning
study = optuna.create_study(direction="minimize")
obj = Objective(
    model=model,
    dataset_name=dataset_name,
    X=X
    )
study.optimize(
    obj,
    n_trials=n_trials,
)
trial = study.best_trial

# print results
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
print(f'Runtime: {time.perf_counter() - start_time}')

# unpack params for next runs
if model == 'feedforward':
    trial.params["num_hidden_dimensions"] = [ trial.params[f"hidden_dim_{i}"] for i in range(trial.params["num_layers"]) ]
elif model == 'transformer':
    trial.params["model_dim"] = trial.params["model_dim_num_heads_pair"][0]
    trial.params["num_heads"] = trial.params["model_dim_num_heads_pair"][1]

# repeat best run 5 times
mases = []
smapes = []
params_sets = []
save_dir = f'results/monash_gluonts_single_tuned_runs/{model}_{dataset_name}_{save_label}_{n_trials}_trials'
os.makedirs(save_dir, exist_ok=True)
for i in range(n_repeats):
    res, predictor, history = obj.train_test(trial.params, tuning=False)

    # plot and save training history
    plt.plot(history.loss_history, label='Training Loss')
    plt.plot(history.validation_loss_history, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Learning Curve')
    plt.legend()
    plt.grid(True)

    # Save the figure
    plt.savefig(f'{save_dir}/learning_curve_{i}.png')
    # save the history values
    with open(f'{save_dir}/loss_history_{i}.json', "w") as f:
        json.dump(history.loss_history, f)
    # Clear the current figure
    plt.clf()
    mases.append(res['MASE'])
    smapes.append(res['sMAPE'])
    params_sets.append(trial.params)


mase_mean = np.array(mases).mean()
mase_std = np.std(np.array(mases))
smape_mean = np.array(smapes).mean()
smape_std = np.std(np.array(smapes))

print(f'##### MASE MEAN: {mase_mean} MASE STD: {mase_std}')
print(f'##### sMAPE MEAN: {smape_mean} sMAPE STD: {smape_std}')

trial.params["mase_mean"] = mase_mean
trial.params["mase_std"] = mase_std
trial.params["smape_mean"] = smape_mean
trial.params["smape_std"] = smape_std
# save best params to json
with open(f'{save_dir}/params.json', "w") as f:
    json.dump(trial.params, f)

# save the last predictor
os.makedirs(f'{save_dir}/predictor', exist_ok=True)
predictor.serialize(Path(f"{save_dir}/predictor"))

end_time = time.perf_counter()
runtime = (end_time - start_time) / 60